In [ ]:
import time
import polars as pl
import matplotlib.pyplot as plt
import blitzbeaver

In [ ]:
csv_path = "/home/plouc314/Documents/epfl/ma1/shs/project/data/csv"
n_year = 10
dfs = []

for year in range(n_year):
    year += 1835
    df = pl.read_csv(f"{csv_path}/{year}.csv")
    dfs.append(df)

df1 = dfs[0]
df2 = dfs[1]
df3 = dfs[2]

In [ ]:
n_run = 1000

st = time.perf_counter()
r = blitzbeaver.benchmark_distance_functions(
    df1["nom_rue_norm"],
    "alexandre",
    num_runs=n_run,
    distance_function="lv_opti",
)
d = time.perf_counter() - st
print(d)


In [ ]:
def benchmark_dist_comp_pass(
    values1: pl.Series,
    values2: pl.Series,
    cache_dist_threshold: int,
    num_runs: int=1,
    distance_function: str="lv_opti",
) -> tuple[float, float, int]:
    
    r = blitzbeaver.benchmark_feature_distance_calculator(
        values1,
        values2,
        num_runs=num_runs,
        cache_dist_threshold=cache_dist_threshold,
        distance_function=distance_function,
    )
    ns, comp, cache_hit, cache_size, _ = r
    duration = ns / 1_000_000_000
    cache_hit_ratio = cache_hit / comp

    return duration, cache_hit_ratio, cache_size

In [ ]:
col = "chef_prenom_norm"

n_run = 1
cache_dist_threshold = 4

r = blitzbeaver.benchmark_feature_distance_calculator(
    df1[col],
    df2[col],
    num_runs=n_run,
    cache_dist_threshold=cache_dist_threshold,
    distance_function="lv_opti",
)
ns, comp, cache_hit, cache_size, _ = r

duration = ns / 1_000_000_000
cache_hit_ratio = cache_hit / comp

print(f"{duration} sec {cache_hit_ratio*100:.2f}% cache hit cache size: {cache_size}")

In [ ]:
thresholds = [128, 64, 32, 16, 8, 4, 2, 1]

vs1 = df1[col]
vs2 = df2[col]
durations = []
cache_hit_ratios = []
cache_sizes = []

for threshold in thresholds:
    duration, cache_hit_ratio, cache_size = benchmark_dist_comp_pass(vs1, vs2, threshold)
    durations.append(duration)
    cache_hit_ratios.append(cache_hit_ratio)
    cache_sizes.append(cache_size)


In [ ]:
# Create a figure with subplots
fig, axes = plt.subplots(3, 1, figsize=(8, 12))

# Plot each metric against thresholds
axes[0].plot(thresholds, durations, marker='o', linestyle='-', color='b')
axes[0].set_title('Thresholds vs Durations')
axes[0].set_xlabel('Thresholds')
axes[0].set_ylabel('Durations')

axes[1].plot(thresholds, cache_hit_ratios, marker='s', linestyle='-', color='r')
axes[1].set_title('Thresholds vs Cache Hit Ratios')
axes[1].set_xlabel('Thresholds')
axes[1].set_ylabel('Cache Hit Ratios')

axes[2].plot(thresholds, cache_sizes, marker='^', linestyle='-', color='g')
axes[2].set_title('Thresholds vs Cache Sizes')
axes[2].set_xlabel('Thresholds')
axes[2].set_ylabel('Cache Sizes')


plt.tight_layout()
plt.show()

In [ ]:
n_run = 1
cache_dist_threshold = 4

r = blitzbeaver.benchmark_feature_distance_calculator_second_pass(
    df1[col],
    df2[col],
    df3[col],
    num_runs=n_run,
    cache_dist_threshold=cache_dist_threshold,
    distance_function="lv_opti",
)
ns, comp, cache_hit, cache_size, _ = r

duration = ns / 1_000_000_000
cache_hit_ratio = cache_hit / comp

print(f"{duration} sec {cache_hit_ratio*100:.2f}% cache hit cache size: {cache_size}")

In [ ]:
n_run = 1
cache_dist_threshold = 4

rs, _ = blitzbeaver.benchmark_feature_distance_calculator_multi_pass(
    [df[col] for df in dfs],
    num_runs=n_run,
    cache_dist_threshold=cache_dist_threshold,
    distance_function="lv_opti",
)
for r in rs:
    ns, comp, cache_hit, cache_size = r

    duration = ns / 1_000_000_000
    cache_hit_ratio = cache_hit / comp

    print(f"{duration} sec {cache_hit_ratio*100:.2f}% cache hit cache size: {cache_size}")